### Med7 Model Example

In [1]:
import spacy
import pandas as pd
med7 = spacy.load("en_core_med7_lg")

### Example Text

In [17]:
text = 'A patient was prescribed Magnesium hydroxide 400mg/5ml suspension PO of total 30ml bid for the next 5 days.'
doc = med7(text)
[{ent.label_: ent.text} for ent in doc.ents]

[{'DRUG': 'Magnesium hydroxide'},
 {'DOSAGE': '400mg/5ml'},
 {'FORM': 'suspension'},
 {'ROUTE': 'PO'},
 {'DOSAGE': '30ml'},
 {'FREQUENCY': 'bid'}]

### Synthesized dataset

In [37]:
df = pd.read_csv("C:\\Users\\steve\\Downloads\\patient_notes.csv")
df.head()

,Label,Text
0,Metformin 500mg,Patient is advised to take Metformin 500mg onc...
1,Lisinopril 10mg,Lisinopril 10mg prescribed for hypertension; p...
2,Amoxicillin 250mg,Prescribed Amoxicillin 250mg three times a day...
3,Atorvastatin 20mg,Atorvastatin 20mg recommended for cholesterol ...
4,Albuterol 90mcg,"Albuterol 90mcg inhaler, use as needed for ast..."


In [59]:
df['Text'][0]

'Patient is advised to take Metformin 500mg once daily to manage blood sugar levels.'

In [47]:
def extract_drug_dose(text:str) -> list:
    """
    Function inputs text field into med 7 and extracts medication and dose

    Params:
    text: str unstructured text field containing medications perscribed to patient
    """
    res = med7(text)
    output = {ent.label_:ent.text for ent in res.ents if ent.label_ in ['STRENGTH', 'DRUG']}
    return output

In [54]:
df_tidy = (df 
  .assign(output = lambda x: x['Text'].apply(lambda y: extract_drug_dose(y)),
          drug = lambda x: x['output'].apply(lambda y: y.get('DRUG')),
          strength = lambda x: x['output'].apply(lambda y: y.get('STRENGTH')),
          prediction = lambda x: x['drug'] + ' ' +x['strength'],
          result = lambda x: x['Label'] == x['prediction']
  )
 )

df_tidy.head()


,Label,Text,output,drug,strength,prediction,result
0,Metformin 500mg,Patient is advised to take Metformin 500mg onc...,"{'DRUG': 'Metformin', 'STRENGTH': '500mg'}",Metformin,500mg,Metformin 500mg,True
1,Lisinopril 10mg,Lisinopril 10mg prescribed for hypertension; p...,"{'DRUG': 'Lisinopril', 'STRENGTH': '10mg'}",Lisinopril,10mg,Lisinopril 10mg,True
2,Amoxicillin 250mg,Prescribed Amoxicillin 250mg three times a day...,"{'DRUG': 'Amoxicillin', 'STRENGTH': '250mg'}",Amoxicillin,250mg,Amoxicillin 250mg,True
3,Atorvastatin 20mg,Atorvastatin 20mg recommended for cholesterol ...,"{'DRUG': 'Atorvastatin', 'STRENGTH': '20mg'}",Atorvastatin,20mg,Atorvastatin 20mg,True
4,Albuterol 90mcg,"Albuterol 90mcg inhaler, use as needed for ast...","{'DRUG': 'Albuterol', 'STRENGTH': '90mcg'}",Albuterol,90mcg,Albuterol 90mcg,True


In [58]:
print(f"Accuracy = {df_tidy['result'].mean()}")

Accuracy = 0.95
